In [13]:
import sympy as sp 
import numpy as np

In [14]:
import spreadingcode as sc
import math 

sampling_rate = 10e6 # Hz 

# create lookup table for prns  

def sample_sequence(prn_sequence, sampling_rate):
    # Determine the number of samples per chip
    #samples_per_chip = int(sampling_rate / len(prn_sequence))
    chip_rate = 1.023e6 # chips/1s 
    samples_per_chip = int(sampling_rate/ chip_rate)

    # Repeat each element of the PRN sequence 'samples_per_chip' times
    sampled_sequence = np.repeat(prn_sequence, samples_per_chip)
    
    # encode into -1s and 1s 
    sampled_sequence[np.nonzero(sampled_sequence)] = -2
    sampled_sequence = sampled_sequence + 1

    return sampled_sequence

# Assuming you already have the PRN sequence generated and stored in 'prn_sequence'
prn_sequence = sc.PRN(1)

# Sample the PRN sequence at 2 times the PRN chip rate 
sampled_sequence = sample_sequence(prn_sequence, sampling_rate)

# get sequence at this delay 
# TODO I think they encode 1s to -1s and 0s to 1s 
def get_x(rem_tot):
    Tc = 1e-3 # code period is 1 ms 
    index = math.floor(rem_tot/Tc * len(sampled_sequence))
    x_tau_seq = np.roll(sampled_sequence, -1*index) # here, we're 9207 samples in, so make this the start and shift the earlier info to the end  
    
    return x_tau_seq

In [21]:
# simulate measurements
sim_time = 6000 # ms for simulation time 
Ts = (10*1.023e6)**-1 # Hz sample rate

# initials for position and signal quality 
c = 2.99792458e8 # speed of light in m/s 
L1 = 1575.42e6  # L1 carrier frequency 
lam = c/L1 # carrier wavelength 
CN0dBHz = -10 # carrier to noise density ratio dBHz
CN0 = 10**(CN0dBHz/10); # carrier to noise density ratio
R = 1/CN0 # variance of measurment noise 
sig_y = R**.5 # SD of measurement noise 
PdBHz = -130 # signal power dB/Hz
P = 10**(PdBHz/10)
n = 1 # inital noise 
r_r = 0 # on earth's surface ENU
v_r = 1  # m/s  
r_sv = 22000e3 # meters ENU 
dt_sv = 1e-3
sig_v = .1 # m/s 

# iono and tropo
tau_I = 30* 60 # about a 30 minute time constant  
tau_T = 2*3600   # about a 2 hour time constant  
sig_I = 1.5 # should be about between 1-5 meters 
sig_T = .5 # should be rougly 1/2 of Iono 
I = 1 # meters
T = .5 # meters

# accelerometer with bias 
sig_a = 57e-6 # SD accelerometer noise m/s^1.5 - > square it times Ts to get m^2/s^2 = discrete accelerometer noise variance   
sig_b = 14e-6  # (m/s^2) continuous SD of accelerometer bias RW noise
tau_b = 3600 # s time constant of accelerometer bias RW    
b = 14e-6 # initial guess at bias 

# clock TXCO 
sig_d = 5e-8 * (Ts)**.5 # 5e-8*sqrt(s) was the given spec, mult by sqrt Ts to get seconds for clock drift 
b_c = 1e-8 # initial receiver clock bias 
d_c = 2*sig_d # initial receiver clock drift 

# store truth data 
state_out = []
state_out.append([I,T,b,v_r,r_r])
signal_out = []
for i in range(0, sim_time):
    # propagate states 
    I = I + Ts*(-1/tau_I * I + np.random.normal(0, sig_I)) # Ionospheric position offset 
    T = T + Ts*(-1/tau_T * T + np.random.normal(0, sig_T)) # Tropospheric position offset 
    b = b + Ts*(-1/tau_b * b + np.random.normal(0, sig_b)) # accelerometer bias 
    v_r = v_r + np.random.normal(0, sig_v) # velocity 
    r_r = r_r + Ts*v_r # new position
    w_d = np.random.normal(0, sig_d) # clock noise 
    b_c = b_c + d_c*Ts + .5*w_d*Ts # bk + dkTs + .5 w Ts^2 clock bias  
    d_c = d_c + w_d  # clock drift
    r = abs(r_r - r_sv) # r_r is the 3d user position, r_sv is 3d space vehicle position from ephemeris
    state_out.append([I,T,b,v_r,r_r])

    # signal params 
    tau = 1/c*(r + I + T + c*(b_c + dt_sv)) # time of transit 
    tau_r = tau % 1 # get decimal part of transit time in seconds 
    phase = 2*np.pi/lam * (r - I + T + c*(b_c + dt_sv))
    # x_t = get_x(tau_r)
    # we will store the parts of the signal so as to not overflow data 
    signal_out.append([(2*P)**.5, tau_r, phase])

# intial guesses and covariance
sig_r_hat = 5 # m 
sig_I_hat = 2 # m 
sig_T_hat = 2 # m
sig_d_clk = sig_d*4 # s 
sig_b_a_hat = .1 # g
sig_a_hat = 1e-4 # g 

r_0 = 4 # m
r_hat = abs(4 - r_sv)
I_hat = 3 
T_hat = I_hat/2 


In [24]:
import sympy as sp
# define symbolic I and Q 

# initials for position and signal quality 
c = 2.99792458e8 # speed of light in m/s 
L1 = 1575.42e6  # L1 carrier frequency 
lam = c/L1 # carrier wavelength 

# Define the symbols
a = sp.symbols('a')
epsilon = sp.symbols('epsilon')  # argument to R functions
R = sp.Function('R')(epsilon)
R_prime = sp.Function('R_prime')(epsilon)
r_r = sp.Matrix([sp.symbols('r_r1 r_r2 r_r3')])
v_r = sp.Matrix([sp.symbols('v_r1 v_r2 v_r3')])
r_sv = sp.Matrix([sp.symbols('r_sv1 r_sv2 r_sv3')])
v_sv = sp.Matrix([sp.symbols('v_sv1 v_sv2 v_sv3')])

# shared line of sight vector 
dr = (r_r - r_sv)
r_los = (r_r - r_sv) / sp.sqrt(dr.dot(dr)) 

# states are in order: a, r_r, I, T, dt_r 
# Define the terms for y_I
term1_I = R
term2_I = a/c * R_prime *  r_los.dot((v_r - v_sv))
term3_I = a * R_prime / c
term4_I = term3_I  # since term3 and term4 are the same
term5_I = term3_I  # same as term3 and term4

# define the terms for y_Q 
term1_Q = 0
term2_Q = 2*sp.pi*a/lam * R * r_los.dot((v_r - v_sv))
term3_Q = -2*sp.pi*a/lam * R
term4_Q = -1*term3_Q
term5_Q = term4_Q 

h_lin_I = sp.Matrix([term1_I, term2_I, term3_I, term4_I, term5_I])
h_lin_Q = sp.Matrix([term1_Q, term2_Q, term3_Q, term4_Q, term5_Q])

# Lambdify
variables = [a, R, R_prime, r_r[0], r_r[1], r_r[2], v_r[0], v_r[1], v_r[2]]
h_func_I = sp.lambdify(variables, h_lin_I, "numpy")
h_func_Q = sp.lambdify(variables, h_lin_Q, "numpy")

In [10]:
import sympy as sp
# define symbolic I and Q for 1D case  

# initials for position and signal quality 
c = 2.99792458e8 # speed of light in m/s 
L1 = 1575.42e6  # L1 carrier frequency 
lam = c/L1 # carrier wavelength 

# Define the symbols
a = sp.symbols('a')
epsilon = sp.symbols('epsilon')  # argument to R functions
R = sp.Function('R')(epsilon)
R_prime = sp.Function('R_prime')(epsilon)

# Define symbolic component
r_r1 = sp.symbols('r_r1')
v_r1 = sp.symbols('v_r1')

# Create vector with symbolic and constant components
r_r = sp.Matrix([[r_r1], [0], [0]])
v_r = sp.Matrix([[v_r1], [0], [0]])

# SV position remains totally symbolic 
r_sv = sp.Matrix([[sp.symbols('r_sv1')], [sp.symbols('r_sv2')], [sp.symbols('r_sv3')]])
v_sv = sp.Matrix([[sp.symbols('v_sv1')], [sp.symbols('v_sv2')], [sp.symbols('v_sv3')]])

# define R and R' 
epsilon = sp.symbols('epsilon')
Tc = 1/1.023e6  # chip duration
# Define R(epsilon)
R_epsilon = sp.Piecewise((1 - abs(epsilon)/Tc, abs(epsilon) < Tc), (0, True))
# Define R'(epsilon)
R_prime_epsilon = sp.Piecewise((1.023e6, epsilon < 0), (-1.023e6, epsilon > 0), (0, True))
# Lambdify
R_func = sp.lambdify(epsilon, R_epsilon, "numpy")
R_prime_func = sp.lambdify(epsilon, R_prime_epsilon, "numpy")

# shared line of sight vector 
dr = (r_r - r_sv)
range_los = sp.sqrt(dr.dot(dr))

# states are in order: a, r_r, I, T, dt_r 
# Define the terms for y_I
term1_I = R
term2_I = a/c * R_prime *  abs(r_r[0]-r_sv[0]) / range_los
term3_I = a * R_prime / c
term4_I = term3_I  # since term3 and term4 are the same
term5_I = term3_I  # same as term3 and term4

# define the terms for y_Q 
term1_Q = 0
term2_Q = 2*sp.pi*a/lam * R *  abs(r_r[0]-r_sv[0]) / range_los
term3_Q = -2*sp.pi*a/lam * R
term4_Q = -1*term3_Q
term5_Q = term4_Q 

h_lin_I = sp.Matrix([term1_I, term2_I, term3_I, term4_I, term5_I])
h_lin_Q = sp.Matrix([term1_Q, term2_Q, term3_Q, term4_Q, term5_Q])

# Lambdify
variables = [a, R, R_prime, r_r1, v_r1, r_sv[0], r_sv[1], r_sv[2], v_sv[0], v_sv[1], v_sv[2]]
h_func_I = sp.lambdify(variables, h_lin_I, "numpy")
h_func_Q = sp.lambdify(variables, h_lin_Q, "numpy")

0.5
-1023000.0


In [ ]:
# load in an I or Q 
# y_e_I = 1/N SUM(I*h_e_I)
PdBHz = -130 # signal power dB/Hz
P = 10**(PdBHz/10)
h_e = h_func_I(np.sqrt(2*P), )